In [1]:
import scipy.signal
import sys
import torch
import torch.nn as nn
import sys
sys.path.append("/home/xhnfly/Cosmic_rays_X/XRL/")
import numpy as np

In [2]:
import math
import random
from copy import deepcopy
import torch
from torch.optim import Adam
from torch.optim import RMSprop
import XRL_main
import gym
import time
from collections import namedtuple, deque
import neptune.new as neptune
import datetime

# from tqdm import tqdm

In [3]:
nep_log = neptune.init(
    project="xhnfirst/DDPG-mian-test1",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI1NTg5MDI2OS01MTVmLTQ2YjUtODA1Yy02ZWQyNDgxZDcwN2UifQ==",
)

https://app.neptune.ai/xhnfirst/DDPG-mian-test1/e/DDPGMIAN-86
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#.stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


In [4]:
import math
import sys
from time import strftime
from time import gmtime

class view_bar:
    def __init__(self, message, num, total, value_name, value_data, start_time_rec, r_true, reward_ctrl, reward_dist, total_main):
        self.message = message
        self.num = num
        self.total = total
        self.message2 = "True time of r = "
        self.r_true = r_true
        self.message3 = "time cost = "
        rate = self.num  / self.total
        rate_num = int(rate * 40)
        rate_nums = math.ceil(rate * 100)
        time_rec = datetime.datetime.now()
        execution_time = time_rec - start_time_rec
        self.message4 = "{reward_ctrl: "
        self.reward_ctrl = reward_ctrl
        self.message5 = "reward_dist: "
        self.reward_dist = reward_dist
        self.message6 = "Total Reward Main: "
        self.total_main = total_main
        self.message8 = " }"



        r = '\r%s:[%s%s]%d%%\t%d/%d\t%s%f\t%s%s\t%s%s\t\t\t\t%s%.4f\t%s%.4f\t%s%.4f\t%s' % (self.message, "=" * rate_num,
                                        " " * (40 - rate_num), rate_nums, self.num , self.total, value_name, value_data, 
                                        self.message3, execution_time, self.message2, str(self.r_true), str(self.message4), self.reward_ctrl,
                                        str(self.message5), self.reward_dist, str(self.message6), self.total_main, 
                                         str(self.message8)
                                        )


        sys.stdout.write(r)
        sys.stdout.flush()


# a =10
# for i in range(60000):
#     view_bar("epoch ", i+1, 60000, 'a = ', float(a))
#     a += 0.01


In [5]:
env = gym.make('Reacher-v2')
test_env = gym.make('Reacher-v2')
device= torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('device = ', device)

device =  cuda


In [6]:
def mlp(sizes, activation, output_activation=nn.Identity):
    layers = []
    for j in range(len(sizes)-1):
        act = activation if j < len(sizes)-2 else output_activation
        layers += [nn.Linear(sizes[j], sizes[j+1]), act()]
    return nn.Sequential(*layers)


class MLPActor(nn.Module):

    def __init__(self, obs_dim, act_dim, hidden_sizes, activation, act_limit):
        super().__init__()
        pi_sizes = [obs_dim] + list(hidden_sizes) + [act_dim]
        self.pi = mlp(pi_sizes, activation, nn.Tanh)
        self.act_limit = act_limit

    def forward(self, obs):
        # Return output from network scaled to action space limits.
        return self.act_limit * self.pi(obs)

class MLPQFunction(nn.Module):

    def __init__(self, obs_dim, act_dim, hidden_sizes, activation):
        super().__init__()
        self.q = mlp([obs_dim + act_dim] + list(hidden_sizes) + [1], activation)

    def forward(self, obs, act):
        q = self.q(torch.cat([obs, act], dim=-1))
        return torch.squeeze(q, -1) # Critical to ensure q has right shape.

class MLPActorCritic(nn.Module):

    def __init__(self, observation_space, action_space, hidden_sizes=(256,256),
                 activation=nn.ReLU, device=torch.device("cuda" if torch.cuda.is_available() else "cpu")):
        super().__init__()

        obs_dim = observation_space.shape[0]
        act_dim = action_space.shape[0]
        act_limit = action_space.high[0]

        # build policy and value functions
        self.pi = MLPActor(obs_dim, act_dim, hidden_sizes, activation, act_limit).to(device)
        self.q = MLPQFunction(obs_dim, act_dim, hidden_sizes, activation).to(device)

    def act(self, obs):
        with torch.no_grad():
            return self.pi(obs)

In [7]:
Transition = namedtuple('Transition',
                        ('obs', 'act', 'rew', 'next_obs', 'done'))

class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([],maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [10]:

params = {
    "batch_size": 128,
    "dropout": 0.2,
    "learning_rate": 0.001,
    "optimizer": "Adam",
    "hid": 64,
    "l": 3,
    "seed": 0,
    "steps_per_epoch": 3000,
    "epochs": 100,
    "replay_size": int(1e6),
    "gamma": 0.99,
    "polyak": 0.995,
    "pi_lr": 1e-4,
    "q_lr": 1e-4,
    "batch_size": 500,
    "start_steps": 10000, 
    "update_after": 1000,
    "update_every": 50,
    "act_noise": 0.01,
    "num_test_episodes": 10,
    "max_ep_len": 1000
}


ac_kwargs=dict(hidden_sizes=[params["hid"]]*params["l"])



In [8]:
nep_log["parameters"] = params



torch.manual_seed(params["seed"])
np.random.seed(params["seed"])
obs_dim = env.observation_space.shape[0]

print('obs_dim = ', obs_dim)
act_dim = env.action_space.shape[0]
print('act_dim = ', act_dim)
# Action limit for clamping: critically, assumes all dimensions share the same bound!
act_limit = env.action_space.high[0]
print('act_limit = ', act_limit)
# Create actor-critic module and target networks
ac = MLPActorCritic(env.observation_space, env.action_space, **ac_kwargs)
ac_targ = deepcopy(ac)

# Freeze target networks with respect to optimizers (only update via polyak averaging)
for p in ac_targ.parameters():
    p.requires_grad = False

memory = ReplayMemory(params["replay_size"])


# Set up function for computing DDPG Q-loss
def compute_loss_q(data):

    o = torch.cat(data.obs).float()
    a = torch.cat(data.act).float()
    r = torch.cat(data.rew).float()
    o2 =torch.cat(data.next_obs).float()
    d = torch.cat(data.done).float()

    q = ac.q(o,a)


    # Bellman backup for Q function
    with torch.no_grad():
        q_pi_targ = ac_targ.q(o2, ac_targ.pi(o2))
        backup = r + params["gamma"] * (1 - d) * q_pi_targ

    # MSE loss against Bellman backup
    loss_q = ((q - backup)**2).mean()

    return loss_q

# Set up function for computing DDPG pi loss
def compute_loss_pi(data):

    o = torch.cat(data.obs).float()

    q_pi = ac.q(o, ac.pi(o))

    return -q_pi.mean()

pi_optimizer = RMSprop(ac.pi.parameters(), lr=params["pi_lr"])
q_optimizer = RMSprop(ac.q.parameters(), lr=params["q_lr"])

def update(data):
    # First run one gradient descent step for Q.


    q_optimizer.zero_grad()
    loss_q = compute_loss_q(data)

    loss_q.backward()

    q_optimizer.step()


    # Freeze Q-network so you don't waste computational effort 
    # computing gradients for it during the policy learning step.
    for p in ac.q.parameters():
        p.requires_grad = False

    # Next run one gradient descent step for pi.
    pi_optimizer.zero_grad()
    loss_pi = compute_loss_pi(data)
    loss_pi.backward()
    pi_optimizer.step()

    # Unfreeze Q-network so you can optimize it at next DDPG step.
    for p in ac.q.parameters():
        p.requires_grad = True


    # Finally, update target networks by polyak averaging.
    with torch.no_grad():
        for p, p_targ in zip(ac.parameters(), ac_targ.parameters()):
            # NB: We use an in-place operations "mul_", "add_" to update target
            # params, as opposed to "mul" and "add", which would make new tensors.
            p_targ.data.mul_(params["polyak"])
            p_targ.data.add_((1 - params["polyak"]) * p.data)


def get_action(o, noise_scale):
    a = ac.act(torch.tensor(o, dtype=torch.float32, device=device))
    a += noise_scale * torch.randn(act_dim).to(device)
    return torch.clip(a, -act_limit, act_limit)

def test_agent():
    for j in range(params["num_test_episodes"]):
        o, d, ep_ret, ep_len = test_env.reset(), False, 0, 0
        if j == 9:
            test_env.render()
            time.sleep(0.02)
        while not(d or (ep_len == params["max_ep_len"])):
            # Take deterministic actions at test time (noise_scale=0)
            a_cpu = get_action(o, 0).cpu().data.numpy()
            # a = get_action(o, 0)
            if j == 9:
                test_env.render()
                time.sleep(0.02)
            o, r, d, _ = test_env.step(a_cpu)
            if d == True:
                r += 10
            else:
                r += -1
            ep_ret += r
            ep_len += 1
        
        nep_log["test/reward"].log(ep_ret/ep_len)
        


# Prepare for interaction with environment
total_steps = params["steps_per_epoch"] * params["epochs"]
start_time = time.time()
o, ep_ret, ep_len = env.reset(), 0, 0
o = torch.tensor([o], device=device)

start_time_rec = datetime.datetime.now()
# Main loop: collect experience in env and update/log each epoch
r_true = 0
total_main = 0
reward_dict={}
ep_times = 0
for t in range(total_steps):
    
    # Until start_steps have elapsed, randomly sample actions
    # from a uniform distribution for better exploration. Afterwards, 
    # use the learned policy (with some noise, via act_noise). 
    if t > params["start_steps"]:
        a = get_action(o, params["act_noise"])
        # a_cpu = a
        a_cpu = a.cpu().data.numpy()
        
    else:
        a = env.action_space.sample()
        a_cpu = a
        a = torch.tensor([a], device=device)
    

    # Step the env
    o2, r, d, reward_dict= env.step(a_cpu)

    nep_log["testreward/dist"].log(reward_dict["reward_dist"])
    nep_log["testreward/ctrl"].log(reward_dict["reward_ctrl"])
        


    if np.abs(reward_dict["reward_dist"]) < 0.1 and np.abs(reward_dict["reward_ctrl"]) < 1.5:
        r += 2
        if np.abs(reward_dict["reward_dist"]) < 0.05 and np.abs(reward_dict["reward_ctrl"]) < 1:
            r += 4
            if np.abs(reward_dict["reward_dist"]) < 0.03 and np.abs(reward_dict["reward_ctrl"]) < 0.7:
                r += 8
                if np.abs(reward_dict["reward_dist"]) < 0.02 and np.abs(reward_dict["reward_ctrl"]) < 0.4:
                    r += 16
                    if np.abs(reward_dict["reward_dist"]) < 0.01 and np.abs(reward_dict["reward_ctrl"]) < 0.2:
                        r += 32
                        if np.abs(reward_dict["reward_dist"]) < 0.005 and np.abs(reward_dict["reward_ctrl"]) < 0.05:
                            r += 64
                            if np.abs(reward_dict["reward_dist"]) < 0.001 and np.abs(reward_dict["reward_ctrl"]) < 0.01:
                                r += 128
                                if np.abs(reward_dict["reward_dist"]) < 0.0001 and np.abs(reward_dict["reward_ctrl"]) < 0.001:
                                    r += 256
        r_true += 1
    else:
        r += -0.1


    
    ep_len += 1
    total_main += r


    # Ignore the "done" signal if it comes from hitting the time
    # horizon (that is, when it's an artificial terminal signal
    # that isn't based on the agent's state)
    d = False if ep_len==params["max_ep_len"] else d

    # a = torch.tensor([a], device=device)
    o2 = torch.tensor([o2], device=device)
    r = torch.tensor([r], device=device)
    d = torch.tensor([d], device=device)

    # Store experience to replay buffer
    memory.push(o, a, r, o2, d)
    # Super critical, easy to overlook step: make sure to update 
    # most recent observation!
    o=o2

    # End of trajectory handling
    if d or (ep_len == params["max_ep_len"]):
        o, ep_ret, ep_len = env.reset(), 0, 0
        ep_times += 1
        ep_ret += r
        ep_ret_main = ep_ret/ep_times
        o = torch.tensor([o], device=device)


    # Update handling
    if t >= params["update_after"] and t % params["update_every"] == 0:
        view_bar("steps: ", t, total_steps, 'main of rewards = ', float(ep_ret_main), start_time_rec, 
                r_true, reward_dict["reward_ctrl"], reward_dict["reward_dist"], total_main/t)
        
        nep_log["train/reward"].log(ep_ret_main)
        for i in range(params["update_every"]):

            transitions = memory.sample(params["batch_size"])
            # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
            # detailed explanation). This converts batch-array of Transitions
            # to Transition of batch-arrays.
            batch = Transition(*zip(*transitions))

            # print('batch = ', str(batch))
            update(data=batch)

    # End of epoch handling
    if (t+1) % params["steps_per_epoch"] == 0:
        epoch = (t+1) // params["steps_per_epoch"]

        # Test the performance of the deterministic version of the agent.
        test_agent()



obs_dim =  11
act_dim =  2
act_limit =  1.0


/tmp/ipykernel_3054/2016876372.py:156: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /opt/conda/conda-bld/pytorch_1639180588308/work/torch/csrc/utils/tensor_new.cpp:201.)
  o = torch.tensor([o], device=device)


steps: :[                                        ]1%	2950/300000	main of rewards = -0.014855	time cost = 0:00:12.772520	True time of r = 483				{reward_ctrl: -0.1154	reward_dist: -0.1239	Total Reward Main: -0.2498	 }Creating window glfw
steps: :[=                                       ]4%	10050/300000	main of rewards = -0.011407	time cost = 0:01:01.304341	True time of r = 1637				{reward_ctrl: -1.9587	reward_dist: -0.3088	Total Reward Main: -0.3341	 }

/tmp/ipykernel_3054/2016876372.py:123: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  a = ac.act(torch.tensor(o, dtype=torch.float32, device=device))


steps: :[======================================= ]100%	299950/300000	main of rewards = 0.000111	time cost = 0:45:06.964054	True time of r = 125022				{reward_ctrl: -1.3163	reward_dist: -0.1447	Total Reward Main: 3.3620	 }}

In [9]:
nep_log.stop()

Shutting down background jobs, please wait a moment...
Done!


Waiting for the remaining 11 operations to synchronize with Neptune. Do not kill this process.


All 11 operations synced, thanks for waiting!


In [14]:
model = ac.q
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

Model's state_dict:
q.0.weight 	 torch.Size([64, 13])
q.0.bias 	 torch.Size([64])
q.2.weight 	 torch.Size([64, 64])
q.2.bias 	 torch.Size([64])
q.4.weight 	 torch.Size([64, 64])
q.4.bias 	 torch.Size([64])
q.6.weight 	 torch.Size([1, 64])
q.6.bias 	 torch.Size([1])


In [15]:
model = ac.pi
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

Model's state_dict:
pi.0.weight 	 torch.Size([64, 11])
pi.0.bias 	 torch.Size([64])
pi.2.weight 	 torch.Size([64, 64])
pi.2.bias 	 torch.Size([64])
pi.4.weight 	 torch.Size([64, 64])
pi.4.bias 	 torch.Size([64])
pi.6.weight 	 torch.Size([2, 64])
pi.6.bias 	 torch.Size([2])


In [16]:
print("q_optimizer's state_dict:")
for var_name in q_optimizer.state_dict():
    print(var_name, "\t", q_optimizer.state_dict()[var_name])

Optimizer's state_dict:
state 	 {0: {'step': 299000, 'square_avg': tensor([[4.0154e+04, 1.1398e+04, 1.9016e+04, 4.6284e+04, 2.8081e+02, 1.0971e+03,
         2.0036e+05, 2.8146e+05, 1.8131e+02, 1.0009e+02, 0.0000e+00, 3.9916e+03,
         5.3032e+03],
        [1.9227e+04, 8.7835e+03, 9.0269e+03, 2.2230e+04, 2.4469e+02, 5.8056e+02,
         2.6806e+05, 3.4891e+05, 1.1042e+02, 9.7425e+01, 0.0000e+00, 4.4064e+03,
         4.4301e+03],
        [4.9201e+04, 5.8895e+03, 2.8196e+04, 6.9876e+04, 2.6348e+02, 2.2326e+03,
         1.2111e+05, 1.3530e+05, 5.2330e+01, 8.1701e+01, 0.0000e+00, 4.0982e+03,
         2.1896e+03],
        [2.6618e+04, 1.3230e+04, 2.1352e+04, 3.2111e+04, 3.8194e+02, 1.0718e+03,
         4.1354e+05, 3.6512e+05, 1.3061e+02, 2.5285e+02, 0.0000e+00, 5.2890e+03,
         4.9570e+03],
        [1.4808e+04, 5.8238e+03, 9.3255e+03, 1.7588e+04, 1.3935e+02, 5.6243e+02,
         1.8936e+05, 1.8219e+05, 1.0091e+02, 7.9645e+01, 0.0000e+00, 2.4438e+03,
         3.0534e+03],
        [1.51

In [17]:
print("pi_optimizer's state_dict:")
for var_name in pi_optimizer.state_dict():
    print(var_name, "\t", pi_optimizer.state_dict()[var_name])

pi_optimizer's state_dict:
state 	 {0: {'step': 299000, 'square_avg': tensor([[8.6105e-01, 4.9790e-01, 1.8904e-01, 5.4539e-01, 1.1169e-02, 9.4090e-03,
         9.5097e+00, 1.5327e+01, 1.5116e-02, 8.9920e-03, 0.0000e+00],
        [1.4412e+00, 6.7878e-01, 2.6247e-01, 9.7308e-01, 1.9922e-02, 2.0455e-02,
         4.6969e+00, 9.8027e+00, 7.5594e-03, 4.6400e-03, 0.0000e+00],
        [1.4291e+00, 7.7289e-01, 2.2776e-01, 9.4793e-01, 1.6435e-02, 2.0465e-02,
         6.4613e+00, 9.6408e+00, 9.9954e-03, 7.9790e-03, 0.0000e+00],
        [1.4412e-01, 6.4755e-02, 3.0563e-01, 4.5285e-01, 3.6344e-03, 7.4650e-03,
         6.7360e+00, 4.1453e+00, 1.6342e-03, 2.7985e-03, 0.0000e+00],
        [8.4601e-02, 3.6184e-02, 2.1278e-02, 7.0556e-02, 1.1436e-03, 1.3483e-03,
         7.9308e-01, 1.0898e+00, 6.9021e-04, 5.5292e-04, 0.0000e+00],
        [1.2071e+00, 3.8491e-01, 2.7472e-01, 1.0457e+00, 1.0691e-02, 2.4897e-02,
         5.6090e+00, 5.4176e+00, 7.7108e-03, 3.1993e-03, 0.0000e+00],
        [5.3873e-01, 3.6

In [25]:
torch.save({
            'model of ac.q': ac.q.state_dict(),
            'model of ac.pi': ac.pi.state_dict(),
            'q_optimizer_state_dict': q_optimizer.state_dict(),
            'pi_optimizer_state_dict': pi_optimizer.state_dict(),
            
            }, "model_nn/model_nn2.pt")

In [11]:
ac = MLPActorCritic(env.observation_space, env.action_space, **ac_kwargs)

In [13]:
# Initialize model
model_q = ac.q
model_pi = ac.pi
pi_optimizer = RMSprop(ac.pi.parameters(), lr=params["pi_lr"])
q_optimizer = RMSprop(ac.q.parameters(), lr=params["q_lr"])


checkpoint = torch.load("model_nn/model_nn2.pt")
model_q.load_state_dict(checkpoint['model of ac.q'])
model_pi.load_state_dict(checkpoint['model of ac.pi'])
q_optimizer.load_state_dict(checkpoint['q_optimizer_state_dict'])
pi_optimizer.load_state_dict(checkpoint['pi_optimizer_state_dict'])




            

In [14]:
model = ac.q
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

Model's state_dict:
q.0.weight 	 torch.Size([64, 13])
q.0.bias 	 torch.Size([64])
q.2.weight 	 torch.Size([64, 64])
q.2.bias 	 torch.Size([64])
q.4.weight 	 torch.Size([64, 64])
q.4.bias 	 torch.Size([64])
q.6.weight 	 torch.Size([1, 64])
q.6.bias 	 torch.Size([1])


In [15]:
model = ac.pi
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

Model's state_dict:
pi.0.weight 	 torch.Size([64, 11])
pi.0.bias 	 torch.Size([64])
pi.2.weight 	 torch.Size([64, 64])
pi.2.bias 	 torch.Size([64])
pi.4.weight 	 torch.Size([64, 64])
pi.4.bias 	 torch.Size([64])
pi.6.weight 	 torch.Size([2, 64])
pi.6.bias 	 torch.Size([2])


In [16]:
print("q_optimizer's state_dict:")
for var_name in q_optimizer.state_dict():
    print(var_name, "\t", q_optimizer.state_dict()[var_name])

q_optimizer's state_dict:
state 	 {0: {'step': 299000, 'square_avg': tensor([[4.0154e+04, 1.1398e+04, 1.9016e+04, 4.6284e+04, 2.8081e+02, 1.0971e+03,
         2.0036e+05, 2.8146e+05, 1.8131e+02, 1.0009e+02, 0.0000e+00, 3.9916e+03,
         5.3032e+03],
        [1.9227e+04, 8.7835e+03, 9.0269e+03, 2.2230e+04, 2.4469e+02, 5.8056e+02,
         2.6806e+05, 3.4891e+05, 1.1042e+02, 9.7425e+01, 0.0000e+00, 4.4064e+03,
         4.4301e+03],
        [4.9201e+04, 5.8895e+03, 2.8196e+04, 6.9876e+04, 2.6348e+02, 2.2326e+03,
         1.2111e+05, 1.3530e+05, 5.2330e+01, 8.1701e+01, 0.0000e+00, 4.0982e+03,
         2.1896e+03],
        [2.6618e+04, 1.3230e+04, 2.1352e+04, 3.2111e+04, 3.8194e+02, 1.0718e+03,
         4.1354e+05, 3.6512e+05, 1.3061e+02, 2.5285e+02, 0.0000e+00, 5.2890e+03,
         4.9570e+03],
        [1.4808e+04, 5.8238e+03, 9.3255e+03, 1.7588e+04, 1.3935e+02, 5.6243e+02,
         1.8936e+05, 1.8219e+05, 1.0091e+02, 7.9645e+01, 0.0000e+00, 2.4438e+03,
         3.0534e+03],
        [1.

In [17]:
print("pi_optimizer's state_dict:")
for var_name in pi_optimizer.state_dict():
    print(var_name, "\t", pi_optimizer.state_dict()[var_name])

pi_optimizer's state_dict:
state 	 {0: {'step': 299000, 'square_avg': tensor([[8.6105e-01, 4.9790e-01, 1.8904e-01, 5.4539e-01, 1.1169e-02, 9.4090e-03,
         9.5097e+00, 1.5327e+01, 1.5116e-02, 8.9920e-03, 0.0000e+00],
        [1.4412e+00, 6.7878e-01, 2.6247e-01, 9.7308e-01, 1.9922e-02, 2.0455e-02,
         4.6969e+00, 9.8027e+00, 7.5594e-03, 4.6400e-03, 0.0000e+00],
        [1.4291e+00, 7.7289e-01, 2.2776e-01, 9.4793e-01, 1.6435e-02, 2.0465e-02,
         6.4613e+00, 9.6408e+00, 9.9954e-03, 7.9790e-03, 0.0000e+00],
        [1.4412e-01, 6.4755e-02, 3.0563e-01, 4.5285e-01, 3.6344e-03, 7.4650e-03,
         6.7360e+00, 4.1453e+00, 1.6342e-03, 2.7985e-03, 0.0000e+00],
        [8.4601e-02, 3.6184e-02, 2.1278e-02, 7.0556e-02, 1.1436e-03, 1.3483e-03,
         7.9308e-01, 1.0898e+00, 6.9021e-04, 5.5292e-04, 0.0000e+00],
        [1.2071e+00, 3.8491e-01, 2.7472e-01, 1.0457e+00, 1.0691e-02, 2.4897e-02,
         5.6090e+00, 5.4176e+00, 7.7108e-03, 3.1993e-03, 0.0000e+00],
        [5.3873e-01, 3.6